In [2]:
# to enable table format
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

# to disbale table format
# from google.colab import data_table
# data_table.disable_dataframe_formatter()

!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 16.3 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.2.0 which is incompatible.


In [3]:
# magic interactive shell command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [4]:
# for data exploration
import pandas as pd
import numpy as np
import datetime as dt 

# remove warnings 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

# for text processing 
import re
import nltk 

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import string 
string.punctuation

from bs4 import BeautifulSoup
# from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from wordcloud import WordCloud,STOPWORDS, ImageColorGenerator

# gensim for LDA
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# plotting tools
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as ex
from plotly.subplots import make_subplots
import pyLDAvis
# import pyLDAvis.gensim #dont skip this
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

# to display all columns ana rows
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.min_rows',None)
pd.set_option('display.expand_frame_repr',True)

print('Libraries Imported!')



'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Libraries Imported!


In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [6]:
# upload file locally
# from google.colab import files
# uploaded = files.upload()

# file upload through google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# # read raw data
# dataset = pd.read_csv(io.BytesIO(uploaded['sample_us.tsv'],sep='\t',error_bad_lines=False))
# df = pd.read_csv('/content/sample_us.tsv',sep='\t',error_bad_lines=False)
df = pd.read_csv("drive/MyDrive/datasets/watch_reviews.tsv",sep='\t',error_bad_lines=False)
df.head()
df.shape

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


(960204, 15)

In [8]:
df[['product_title','review_body','product_category']].head()

,product_title,review_body,product_category
0,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Absolutely love this watch! Get compliments al...,Watches
1,Kenneth Cole New York Women's KC4944 Automatic...,I love this watch it keeps time wonderfully.,Watches
2,Ritche 22mm Black Stainless Steel Bracelet Wat...,Scratches,Watches
3,Citizen Men's BM8180-03E Eco-Drive Stainless S...,"It works well on me. However, I found cheaper ...",Watches
4,Orient ER27009B Men's Symphony Automatic Stain...,Beautiful watch face. The band looks nice all...,Watches


In [9]:
# check for time range of dataset
time_range=pd.Series(pd.to_datetime((df.review_date.sort_values(ascending=False)))).dropna()
time_diff=(time_range.iloc[0]-time_range.iloc[-1])
print(f'The dataset has a time period from {time_range.iloc[-1].year} to {time_range.iloc[0].year}, in total of {time_diff}.')

The dataset has a time period from 2001 to 2015, in total of 5261 days 00:00:00.


In [10]:
# check dataset size and number of features
print(f"Dataset contians {df.shape[0]} rows and {df.shape[1]} columns")

Dataset contians 960204 rows and 15 columns


In [11]:
# convert review body column to string datatype
df['review_body'] = df['review_body'].astype('str')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960204 entries, 0 to 960203
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        960204 non-null  object
 1   customer_id        960204 non-null  int64 
 2   review_id          960204 non-null  object
 3   product_id         960204 non-null  object
 4   product_parent     960204 non-null  int64 
 5   product_title      960202 non-null  object
 6   product_category   960204 non-null  object
 7   star_rating        960204 non-null  int64 
 8   helpful_votes      960204 non-null  int64 
 9   total_votes        960204 non-null  int64 
 10  vine               960204 non-null  object
 11  verified_purchase  960204 non-null  object
 12  review_headline    960197 non-null  object
 13  review_body        960204 non-null  object
 14  review_date        960200 non-null  object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


**Building Up Text Corpus**

In [12]:
# filter() --> use to filter data from the dataset
# finding missing values
df['review_id'].isnull().sum()


0

Considering only those popular products which have more than 300 reviews 

In [13]:
# finding popular products using group by and filtering data where reviews > 300
popular_products = df.groupby(df['product_id']).filter(lambda x: x['review_id'].count()>=300)
print(f'Number of datapoints matching the criteria is: {len(popular_products)}')
data = pd.DataFrame(popular_products)
# check null values for the new dataset
popular_products.isnull().sum()

Number of datapoints matching the criteria is: 132407


marketplace          0
customer_id          0
review_id            0
product_id           0
product_parent       0
product_title        0
product_category     0
star_rating          0
helpful_votes        0
total_votes          0
vine                 0
verified_purchase    0
review_headline      0
review_body          0
review_date          0
dtype: int64

**Text Preprocessing**

**Text preprocessing/cleaning steps:**
Lower Casing
The idea is to convert the input text into same casing format so that 'text', 'Text' and 'TEXT' are treated the same way.
Pros: More helpful for text featurization like frequency, TF-IDF as it reducing the duplicates.
Cons: May not be helpful in sentiment analysis when upper cases often refers to anger.


**Removal of Punctuation**
The string.punctuation in Python contains the following punctuations:
!"#$%&'()*+,-./:;<=>?@[\]^_{|}~`


Removal of Stopwords **bold text**
The commonly occuring words in a language that doesn't provides valuable information.


Stemming **bold text**
Convert words to it's word stem, base or root. Porter Stemmer is for English language. If we are working with other languages, we can use snowball stemmer.


Lemmatization **bold text**
Convert the word to it's original form based on its context, slower but more linguistic revalent.
NLTK, Stanford, Bitext Lemmatizer available.


**Removal of emojis**
For social media text corpuses, an explosion usage of emoji should also be taken care of. Functions credit to this repo.
For sentiment analysis or similar purposes, the emotion delivered from emojis are unnegligible, and thanks to this repo, we can make use this dictionary to conver the emojis to corresponding words.
For this project's purposes, since the data ends in 2015 and not as abundant emojis are available by then, we can skip this part of preprocessing.


**Removal of URLs**
When doing Twitter/Reddit analysis, there's good chance of having URLs present in data, we might need to remove it for analysis.


Removal of HTML tags
When data is scrapped from websites, we might end up having html strings as part of the text. Can remove it by using Regex or a more elegant way of using BeautifulSoup.

Stemming

In [14]:
def clean_text_stemming(var):
    """
    Function for text preprocessing with Poter Stemming.
    """
    sw = set(stopwords.words('english'))
    ps = PorterStemmer() 

    # tokenize the word using nltk  
    my_text = nltk.word_tokenize(var)
    
    # remove not english characters, lower case and remove the white space at end 
    my_text = re.sub('[^A-Za-z0-9]+', " ", var).lower().strip()

    # remove the stop words 
    my_text = [word for word in my_text.split() if word not in sw]

    # stemming
    my_text = [ps.stem(word) for word in my_text]

    # removal of URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    my_text = url_pattern.sub(r'', ' '.join(my_text))

    # removal of HTML Tags
    my_text = BeautifulSoup(my_text, "lxml").text

    # convert the text to list as the vectorized words  
    my_text = my_text.split(" ")
    
    return my_text

In [15]:
a = data['review_body'][50:60]
a.apply(lambda x: clean_text_stemming(x))

375                [love, comfort, feel, weak, bracelet]
377                                       [excel, watch]
382          [exactli, need, want, awesom, great, price]
398    [seem, cheap, watch, accur, time, keep, possib...
400    [ok, watch, day, easili, say, best, cheapli, p...
409      [excel, son, love, want, swim, yet, pass, test]
450                                       [pretti, good]
453    [great, watch, great, water, resist, substanti...
497                                         [niec, love]
513    [recommend, watch, high, qualiti, perform, adv...
Name: review_body, dtype: object

Lemmatization

In [16]:
def clean_text_lemma(var):
    """
    Function for text preprocessing with Lemmatizing with POS tag.
    """
    sw = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    wordnet_map = {"N": wordnet.NOUN,"V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
    
    # tokenize the word using nltk  
    my_text = nltk.word_tokenize(var)
    
    # remove not english characters, lower case and split the text 
    my_text = re.sub('[^A-Za-z0-9]+', " ", var).lower().strip() 

    # remove the stop words 
    my_text = [word for word in my_text.split() if word not in sw]

    # lemmatizing 
    pos_tagged_text = nltk.pos_tag(my_text) 
    my_text = " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

    # removal of URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    my_text = url_pattern.sub(r'', my_text)

    # removal of HTML Tags
    my_text = BeautifulSoup(my_text, "lxml").text

    # convert the text to list as the vectorized words  
    my_text = my_text.split(" ")

    return my_text

In [17]:
b = data['review_body'][50:60]
b.apply(lambda var: clean_text_lemma(var))

375            [love, comfortable, feel, weak, bracelet]
377                                   [excellent, watch]
382         [exactly, need, want, awesome, great, price]
398    [seem, cheap, watch, accurate, time, keep, pos...
400    [ok, watch, day, easily, say, best, cheaply, p...
409    [excellent, son, love, want, swim, yet, pass, ...
450                                       [pretty, good]
453    [great, watch, great, water, resistance, subst...
497                                        [niece, love]
513    [recommend, watch, high, quality, perform, adv...
Name: review_body, dtype: object

TF-IDF Tokenization

In [18]:
tfidf_model = TfidfVectorizer(max_df=0.99,
                max_features=1000,
                min_df = 0.01,
                stop_words='english',
                use_idf=True,
                tokenizer=clean_text_lemma,
                ngram_range=(1,3))

tfidf_matrix = tfidf_model.fit_transform(data['review_body'])
print(f"In total there are {tfidf_matrix.shape[0]} review and {tfidf_matrix.shape[1]} terms")



In total there are 132407 review and 386 terms


In [19]:
# save the terms identified by TF-IDF and have a look 
tf_selected_words = tfidf_model.get_feature_names_out()

# print out word snippet 
tf_selected_words[380:]

array(['wrist', 'wrong', 'year', 'year ago', 'year old', 'zone'],
      dtype=object)

In [20]:
tfidf_matrix.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.26266533,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

Now as the text has been transformed into numbers in a way that machine learning algorithm can understand, which is the process as text vectorization, we can feed the TF-IDF score to algorithms for more interesting exploration.

**Part 3: Topic Modeling**


Both K-means and Latent Dirichlet Allocation (LDA) are unsupervised learning algorithms, where the user needs to decide a priori the parameter K, respectively the number of clusters and the number of topics.

If both are applied to assign K topics to a set of N documents, the most evident difference is that K-means is going to partition the N documents in K disjoint clusters (i.e. topics in this case).

On the other hand, LDA assigns a document to a mixture of topics. Therefore each document is characterized by one or more topics (e.g. Document D belongs for 60% to Topic A, 30% to topic B and 10% to topic E). Hence, LDA can give more realistic results than k-means for topic assignment. 

K-Means Clustering

In [21]:
from sklearn.cluster import KMeans

# initialize with 5 clusters
num_clusters = 5
km = KMeans(n_clusters=num_clusters).fit(tfidf_matrix)
clusters = km.labels_.tolist()

Analyze K-means Result

In [22]:
# create DataFrame films from all of the input files.
product = { 'review': data['review_body'], 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])
frame.head(10)

,review,cluster
2,Scratches,2
3,"It works well on me. However, I found cheaper ...",2
26,It works well with nice simple look.,2
29,This watch is a very beautiful time piece. Thi...,4
34,Awesome watch for the price,2
38,This is now my everyday watch. Easy to read w...,0
44,"Have worn it constantly, love the light at night",0
46,Great product . Love the continous innovation ...,2
57,"Alrighty, after seeking out the ultimate low-p...",4
65,"Man watch for woman, just a right size and des...",2


In [23]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
2,87727
3,15575
0,12132
4,11732
1,5241


In [24]:
km.cluster_centers_.shape

(5, 386)

In [28]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
order_centroids

array([[196, 197, 338, ...,  50,  49, 341],
       [137, 246, 142, ..., 341, 265, 383],
       [338, 321, 181, ..., 340,  50, 341],
       [143, 338, 146, ...,  49,  50, 341],
       [ 48,  49, 338, ..., 139, 349,  84]])

The assumption underpins is that using the cluster_center_ to represent the whole cluster, and the higher the TF-IDF value is, the higher representation power of this word is. Therefore, I choose the top 6 words according to their TF-IDF score to represent it's cluster. And see if there's some pattern that we can find from it's clustering result.

In [26]:
# sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for j in order_centroids[i, :6]: # replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(tf_selected_words[j]) # tf_selected_words is a list of the selected words from the text data and is used to retrieve the actual keyword based on the sorted indices from order_centroids
        print (tf_selected_words[j] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews)")
    print ()

Cluster 0 words:love,love watch,watch,gift,buy,husband,
Cluster 0 reviews (12132 reviews)

Cluster 1 words:good,product,good watch,watch,price,good price,
Cluster 1 reviews (5241 reviews)

Cluster 2 words:watch,time,like,nice,look,work,
Cluster 2 reviews (87727 reviews)

Cluster 3 words:great,watch,great watch,price,look,work,
Cluster 3 reviews (15575 reviews)

Cluster 4 words:br,br br,watch,time,look,band,
Cluster 4 reviews (11732 reviews)



Analysis for the K-means clustering results:

From the above clustering results, we can see that the word watch is showing up in each ranking, which is not surprising but redundant this case, for the model imporvement, we could add this domain related word into the stopwords.

Most reviews fell into the cluster 5 with a ratio of 66.24%, yet the tokenized word from it is not so informative, as it just praising the punctuality of the watch.


Business Insight

Cluster 0 though, with the words like love, gift, husband gathering, we can infer that this watch have being giving out as present to the customer's spouse and as a way of showing their affection. I would suggest the brand to have some promotion activities when it comes to Vlentine's Day, and can run an A/B test on whether the promotion would help increase the sales.

**Latent Dirichlet Allocation(LDA)**

Gensim's LDA requires the data in a certain format. Firstly, it needs the corpus as a dicionary of id-word mapping, where each word has a unique numeric ID. This is for computationally efficiency purposes. Secondly, it needs the corpus as a term-document frequency matrix which contains the frequency of each word in each document

Create the dictionary and corpus needed for Topic Modeling

In [29]:
# process all the data using lemmatization
data_lemmatized = data['review_body'].apply(lambda x: clean_text_lemma(x))

In [30]:
# create dictionary
id2word=corpora.Dictionary(data_lemmatized)

#create corpus with Term Document Frequency
corpus=[id2word.doc2bow(text) for text in data_lemmatized]

# sample
print(corpus[2])

[(8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]


Gensim creates a unique id for each word in the document, the produced corpus shown above is a mapping of (word_id, word_frequency).
For instance, (8,1) above implies, word id 8 occurs once in the first document. And this will be used as the input by the LDA model of Gensim.

In [32]:
# human-readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('scratch', 1)]]